In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

In [5]:
torch.cuda.is_available()

False

In [2]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

In [3]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [3]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.edge_weight = torch.nn.Parameter(torch.ones(data.num_edges))
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index,torch.minimum(self.edge_weight.abs(),torch.ones(data.num_edges)))
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index,torch.minimum(self.edge_weight.abs(),torch.ones(data.num_edges)))

        return F.log_softmax(x, dim=1)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [8]:
for i in model.edge_weight:
    print(i)


tensor(0.2750, grad_fn=<UnbindBackward0>)
tensor(0.2550, grad_fn=<UnbindBackward0>)
tensor(0.2224, grad_fn=<UnbindBackward0>)
tensor(0.6024, grad_fn=<UnbindBackward0>)
tensor(0.2829, grad_fn=<UnbindBackward0>)
tensor(0.3227, grad_fn=<UnbindBackward0>)
tensor(0.3555, grad_fn=<UnbindBackward0>)
tensor(0.2263, grad_fn=<UnbindBackward0>)
tensor(0.2948, grad_fn=<UnbindBackward0>)
tensor(0.2389, grad_fn=<UnbindBackward0>)
tensor(0.2219, grad_fn=<UnbindBackward0>)
tensor(0.2858, grad_fn=<UnbindBackward0>)
tensor(0.2510, grad_fn=<UnbindBackward0>)
tensor(0.2564, grad_fn=<UnbindBackward0>)
tensor(0.3029, grad_fn=<UnbindBackward0>)
tensor(0.2869, grad_fn=<UnbindBackward0>)
tensor(0.2709, grad_fn=<UnbindBackward0>)
tensor(0.2716, grad_fn=<UnbindBackward0>)
tensor(0.3052, grad_fn=<UnbindBackward0>)
tensor(0.2585, grad_fn=<UnbindBackward0>)
tensor(0.2268, grad_fn=<UnbindBackward0>)
tensor(0.2161, grad_fn=<UnbindBackward0>)
tensor(0.2199, grad_fn=<UnbindBackward0>)
tensor(0.1898, grad_fn=<UnbindBack

: 

In [7]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7800


In [7]:
for key,param in model.named_parameters():
    print(key)

edge_weight
conv1.bias
conv1.lin.weight
conv2.bias
conv2.lin.weight


In [ ]:
min(model.edge_weight)

In [ ]:
max(model.edge_weight)

In [ ]:
import numpy as np

In [ ]:
a=np.random.randn(10,10)

In [ ]:
a[a>0.5].shape

In [ ]:
torch.sort(model.edge_weight)